In [ ]:
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/VICReg-BarlowTwins-Ablation-Study/autoSSL")
import torchvision
from lightly.data import LightlyDataset, SimCLRCollateFunction, collate
import numpy as np
import tqdm
import yaml
import numpy as np
import torch
from tqdm import tqdm
import pytorch_lightning as pl
from autoSSL.evaluate import eval_KNN,eval_linear,eval_KNNplot
from autoSSL.models import BarlowTwins, BYOL, MoCo, SimCLR, SimSiam, VICReg
from autoSSL.utils import embedding_feature,ck_callback
from autoSSL.data import PipeDataset
from IPython.display import clear_output

In [ ]:
# Load the YAML file
with open('global.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Access the variables from the loaded YAML
num_workers = config['num_workers']
batch_size = config['batch_size']
memory_bank_size = config['memory_bank_size']
seed = config['seed']
device = config['device']
max_epochs = config['max_epochs']
input_size = config['input_size']
path_to_train = config['path_to_train']
path_to_test = config['path_to_test']

#pl.seed_everything(seed)
finetuning_augmentation = {
    'RandomCrop': {'size': input_size, 'padding': 4},
    'RandomHorizontalFlip': {},
    'ToTensor': {},
    'Normalize': {
        'mean': collate.imagenet_normalize["mean"],
        'std': collate.imagenet_normalize["std"]
    }
}
test_augmentation = {
    'ToTensor': {},
    'Normalize': {
        'mean': collate.imagenet_normalize["mean"],
        'std': collate.imagenet_normalize["std"]
    }
}

In [ ]:
# Download and test all,

# Test Dataset

In [ ]:

pipe_data_train =PipeDataset(
    input_dir=path_to_train, 
    #source="cifar10" or "imagenet10",
    #samples=1000, 
    augmentation=test_augmentation, 
    batch_size=batch_size,
    shuffle=True,
    #collate_fn=collate_fn,
    drop_last=True,
    num_workers=num_workers,
 

)

pipe_data_finetuning =PipeDataset(
    input_dir=path_to_train, 
    augmentation=finetuning_augmentation, 
    samples=1000, 
    batch_size=batch_size, 
    shuffle=True,
    drop_last=True,
    num_workers=num_workers,
 

)

pipe_data_test =PipeDataset(
    input_dir=path_to_test, 
    augmentation=test_augmentation, 
    #samples=1000, 
    batch_size=batch_size, 
    shuffle=False,
    drop_last=False,
    num_workers=num_workers,    
 

)

pipe_data_test.plot([2,3,4])
pipe_data_test[2]
len(pipe_data_test)

pipe_data_test.dataloader  # get dataloader 
pipe_data_test.dataset     # get dataset class
pipe_data_test.array       #get array.

clear_output()

In [ ]:
# Test Split

pipe_data_train =PipeDataset(
    input_dir=path_to_train, 
    #samples=1000, 
    augmentation=test_augmentation, 
    batch_size=batch_size,
    shuffle=True,
    #collate_fn=collate_fn,
    drop_last=True,
    num_workers=num_workers,

)
train_pipe_dataset, val_pipe_dataset = pipe_data_train.split(0.8)

print(len(val_pipe_dataset))
print(len(train_pipe_dataset))
print(len(pipe_data_train))

# Test Model

In [ ]:
 
model=MoCo()
model=BYOL()
model=SimCLR()
model=SimSiam()
model=BarlowTwins()
model=VICReg()
model.to(device)
clear_output()

# Test Evaluation

In [ ]:
pipe_data_train =PipeDataset(
    input_dir=path_to_train, 
    #samples=1000, 
    augmentation=test_augmentation, 
    batch_size=batch_size,
    shuffle=True,
    #collate_fn=collate_fn,
    drop_last=True,
    num_workers=num_workers,

)
pipe_data_test =PipeDataset(
    input_dir=path_to_test, 
    #samples=1000, 
    augmentation=test_augmentation, 
    batch_size=batch_size,
    shuffle=True,
    #collate_fn=collate_fn,
    drop_last=True,
    num_workers=num_workers,

)
model=MoCo()
eval_linear(pipe_data_train,embedding_model=model,split=0.8,device="cuda") 
eval_linear(pipe_data_train,embedding_model=model,test=pipe_data_test,device="cuda") 
eval_KNN(pipe_data_train,embedding_model=model,split=0.8,device="cuda") 
eval_KNN(pipe_data_train,embedding_model=model,test=pipe_data_test,device="cuda")   
eval_KNNplot(pipe_data_test,embedding_model=model,samples=2,device="cuda")